In [3]:
# 1. INSTALL & IMPORT

!pip install transformers accelerate timm datasets ultralytics nltk -q

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import BlipProcessor, BlipForConditionalGeneration
from PIL import Image
import pandas as pd
import os
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import nltk
from nltk.translate.bleu_score import sentence_bleu
import json
from ultralytics import YOLO

nltk.download('punkt')


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 18.3 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 94.1 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 60.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 16.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━

2025-09-17 10:26:55.870373: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1758104816.254885      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1758104816.359776      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [4]:
# 2. DEVICE CONFIG

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", DEVICE)

torch.backends.cudnn.enabled = True
torch.backends.cudnn.benchmark = True

Using device: cuda


In [5]:
# 3. PATHS

IMAGES_PATH = "/kaggle/input/flickr30k/Images"
CAPTIONS_FILE = "/kaggle/input/flickr30k/captions.txt"

In [6]:
# 4. LOAD CAPTIONS

df = pd.read_csv(CAPTIONS_FILE)
df.columns = ['image', 'caption']
df = df.dropna()

print("Sample data:")
print(df.head())

Sample data:
            image                                            caption
0  1000092795.jpg   Two young guys with shaggy hair look at their...
1  1000092795.jpg   Two young , White males are outside near many...
2  1000092795.jpg   Two men in green shirts are standing in a yard .
3  1000092795.jpg       A man in a blue shirt standing in a garden .
4  1000092795.jpg            Two friends enjoy time spent together .


In [7]:
# 5. TRAIN / VAL / TEST SPLIT

train_df, temp_df = train_test_split(df, test_size=0.2, random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)
test_df = test_df.sample(n=100, random_state=42)

print(f"Train: {len(train_df)}, Val: {len(val_df)}, Test: {len(test_df)}")


Train: 127131, Val: 15891, Test: 100


In [8]:
 #6. YOLO DETECTION (PRECOMPUTE)

yolo_model = YOLO("yolov8s.pt")  # lightweight model

def detect_objects(image_path):
    results = yolo_model(image_path, verbose=False)
    detections = []
    for r in results:
        for box in r.boxes:
            cls_id = int(box.cls[0])
            label = yolo_model.names[cls_id]
            bbox = box.xyxy[0].cpu().numpy().tolist()  # [x1,y1,x2,y2]
            detections.append({"label": label, "bbox": bbox})
    return detections

# Cache YOLO detections for train/val/test
yolo_detections = {}
for img_name in tqdm(df['image'].unique(), desc="Running YOLO on dataset"):
    img_path = os.path.join(IMAGES_PATH, img_name)
    try:
        yolo_detections[img_name] = detect_objects(img_path)
    except:
        yolo_detections[img_name] = []
with open("yolo_detections.json", "w") as f:
    json.dump(yolo_detections, f)

Running YOLO on dataset: 100%|██████████| 31783/31783 [20:09<00:00, 26.27it/s]  


In [35]:
# 7. DATASET WITH CROPS

class Flickr30kYOLOCropsDataset(Dataset):
    def __init__(self, dataframe, image_path, processor, detections_dict):
        self.dataframe = dataframe
        self.image_path = image_path
        self.processor = processor
        self.detections_dict = detections_dict

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        row = self.dataframe.iloc[idx]
        image_file = os.path.join(self.image_path, row['image'])
        image = Image.open(image_file).convert('RGB')
        caption = row['caption']

        detections = self.detections_dict.get(row['image'], [])
        labels = [d["label"] for d in detections]
        context_text = "Objects detected: " + ", ".join(labels)

        # Crops
        object_images = []
        for d in detections[:3]:  # take up to 3 detections
            x1, y1, x2, y2 = d["bbox"]
            crop = image.crop((x1, y1, x2, y2))
            object_images.append(crop)
        
        # pad with original image until we have exactly 3 crops
        while len(object_images) < 3:
            object_images.append(image)
        
        # now always 4 images: 1 original + 3 crops
        inputs = self.processor(
            images=[image] + object_images,
            text=context_text + " | " + caption,
            return_tensors="pt",
            padding="max_length",
            max_length=50,
            truncation=True
        )

        return {
            "pixel_values": inputs["pixel_values"],  # shape: (N_images, C, H, W)
            "input_ids": inputs["input_ids"].squeeze(),
            "attention_mask": inputs["attention_mask"].squeeze()
        }


In [9]:
# 8. INIT BLIP

processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base").to(DEVICE)


Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


preprocessor_config.json:   0%|          | 0.00/287 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/506 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/990M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

In [36]:
# 9. DATALOADERS 
train_dataset = Flickr30kYOLOCropsDataset(train_df, IMAGES_PATH, processor, yolo_detections)
val_dataset = Flickr30kYOLOCropsDataset(val_df, IMAGES_PATH, processor, yolo_detections)
test_dataset = Flickr30kYOLOCropsDataset(test_df, IMAGES_PATH, processor, yolo_detections) 

train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True, num_workers=2, pin_memory=True) 
val_loader = DataLoader(val_dataset, batch_size=4, shuffle=False, num_workers=2, pin_memory=True)

In [ ]:
# 10. TRAINING

optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)
scaler = torch.amp.GradScaler("cuda")

epochs = 2
for epoch in range(epochs):
    model.train()
    train_loss = 0.0
    loop = tqdm(train_loader, leave=True)
    for batch in loop:
        pixel_values = batch["pixel_values"].to(DEVICE)   # (B, N, 3, H, W)
        input_ids = batch["input_ids"].to(DEVICE)         # (B, seq_len)
        attention_mask = batch["attention_mask"].to(DEVICE)

        # ---- FIX: flatten crops into batch dimension ----
        B, N, C, H, W = pixel_values.shape
        pixel_values = pixel_values.view(B * N, C, H, W)
        input_ids = input_ids.repeat_interleave(N, dim=0)
        attention_mask = attention_mask.repeat_interleave(N, dim=0)
        # -------------------------------------------------

        with torch.autocast("cuda", enabled=True):
            outputs = model(
                pixel_values=pixel_values,
                input_ids=input_ids,
                attention_mask=attention_mask,
                labels=input_ids
            )
            loss = outputs.loss

        optimizer.zero_grad()
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        train_loss += loss.item()
        loop.set_description(f"Epoch [{epoch+1}/{epochs}]")
        loop.set_postfix(loss=loss.item())

    avg_train_loss = train_loss / len(train_loader)
    print(f"Epoch {epoch+1}, Train Loss: {avg_train_loss:.4f}")

    # ---- Validation ----
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for batch in val_loader:
            pixel_values = batch["pixel_values"].to(DEVICE)
            input_ids = batch["input_ids"].to(DEVICE)
            attention_mask = batch["attention_mask"].to(DEVICE)

            # flatten crops in validation too
            B, N, C, H, W = pixel_values.shape
            pixel_values = pixel_values.view(B * N, C, H, W)
            input_ids = input_ids.repeat_interleave(N, dim=0)
            attention_mask = attention_mask.repeat_interleave(N, dim=0)

            with torch.autocast("cuda", enabled=True):
                outputs = model(
                    pixel_values=pixel_values,
                    input_ids=input_ids,
                    attention_mask=attention_mask,
                    labels=input_ids
                )
                val_loss += outputs.loss.item()

    avg_val_loss = val_loss / len(val_loader)
    print(f"Validation Loss: {avg_val_loss:.4f}")



Epoch [1/2]:  32%|███▏      | 10142/31783 [2:51:12<6:03:53,  1.01s/it, loss=0.556]

In [ ]:
# 11. SAVE MODEL

model.save_pretrained("/kaggle/working/blip-yolo-flickr30k")
processor.save_pretrained("/kaggle/working/blip-yolo-flickr30k")
print("Model saved!")

In [ ]:
# 12. TESTING + BLEU

model.eval()
bleu_scores = []

for idx in range(len(test_dataset)):
    data = test_dataset[idx]
    pixel_values = data["pixel_values"].unsqueeze(0).to(DEVICE)

    with torch.no_grad():
        generated_ids = model.generate(pixel_values=pixel_values, max_length=40)
        caption = processor.decode(generated_ids[0], skip_special_tokens=True)

    reference = [test_df.iloc[idx]['caption'].split()]
    candidate = caption.split()
    bleu = sentence_bleu(reference, candidate)
    bleu_scores.append(bleu)

print(f"Average BLEU Score on Test Set: {sum(bleu_scores)/len(bleu_scores):.4f}")
